Preprocessing an input string is the basis needed to do higher-level operations such as tokenizing.

In [1]:
input_str = '༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།'

# pybotextchunks.py

This class is a wrapper around PyBoChunk (that it subclasses).
Its main purpose is to provide the input to be fed into the trie within tokenizer.py

Here is what it produces:

In [2]:
from pybo import PyBoTextChunks
text_chunks = PyBoTextChunks(input_str)

output = text_chunks.serve_syls_to_trie()
print(f'The output is a {type(output)} of {type(output[0])} containing each {len(output[0])} elements.')
print('\tex: ', output[0])
print(f'The first element is either a {type(output[0][0])} or a {type(output[1][0])} of ints')
print('\tex:', output[0][0], 'or', output[1][0])

The output is a <class 'list'> of <class 'tuple'> containing each 2 elements.
	ex:  (None, (102, 0, 2))
The first element is either a <class 'NoneType'> or a <class 'list'> of ints
	ex: None or [2, 3]


See below for the second element, it is the output of PyBoChunk.

In [3]:
for n, trie_input in enumerate(output):
    print(f'{n+1}.\t {trie_input[0]}')

1.	 None
2.	 [2, 3]
3.	 [5, 6, 7]
4.	 [9, 10, 11]
5.	 None
6.	 [18, 19, 20]
7.	 [23, 24, 26, 27]
8.	 None
9.	 [30, 31, 32]
10.	 [34, 35, 36]
11.	 [38, 39, 40]
12.	 [42, 43, 44, 45]
13.	 None
14.	 [50, 51]
15.	 None
16.	 [54, 55, 56, 57]
17.	 [59, 60, 61]
18.	 [63, 64, 65, 66]
19.	 [68, 69, 70]
20.	 [72, 73, 74]
21.	 [76, 77]
22.	 [79, 80]
23.	 [82, 83, 84, 85]
24.	 None
25.	 [92, 93, 94]
26.	 None


The first element provides the information to the trie that a given chunk is a syllable, and thus should be fed to the trie, or that it is not a syllable.

In case it is a syllable, its elements are indices of the characters that constitute the syllable:

In [4]:
print('\t', input_str)
for n, trie_input in enumerate(output):
    syl = trie_input[0]
    if syl:
        syllable = [input_str[s] for s in syl] + ['་']
        print(f'{n+1}.\t{syllable}')
    else:
        print(f'{n+1}.')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།
1.
2.	['ཤ', 'ི', '་']
3.	['བ', 'ཀ', 'ྲ', '་']
4.	['ཤ', 'ི', 'ས', '་']
5.
6.	['བ', 'ད', 'ེ', '་']
7.	['ལ', 'ེ', 'ག', 'ས', '་']
8.
9.	['བ', 'ཀ', 'ྲ', '་']
10.	['ཤ', 'ི', 'ས', '་']
11.	['བ', 'ད', 'ེ', '་']
12.	['ལ', 'ེ', 'ག', 'ས', '་']
13.
14.	['ཀ', 'ཀ', '་']
15.
16.	['མ', 'ཐ', 'འ', 'ི', '་']
17.	['ར', 'ྒ', 'ྱ', '་']
18.	['མ', 'ཚ', 'ོ', 'ར', '་']
19.	['ག', 'ན', 'ས', '་']
20.	['པ', 'འ', 'ི', '་']
21.	['ཉ', 'ས', '་']
22.	['ཆ', 'ུ', '་']
23.	['འ', 'ཐ', 'ུ', 'ང', '་']
24.
25.	['མ', 'ཁ', 'འ', '་']
26.


Punctuation chunks are left aside, non-Tibetan parts also and the content of the syllable has been cleaned.
In token 6, the double tsek has been normalized to a single tsek.
In token 7, the space in the middle has been left aside.
On top of this, the non-syllabic chunks give us a cue that any ongoing word has to end there.

Now, this cleaned content can be fed in the trie, syllable after syllable, character after character.

# pybochunk.py

This class is powered by the chunking framework defined in BoChunk (that it subclasses).

It implements the following chunking pipeline:
 - turn the input string into a sequence of alternating "bo" and "non-bo" chunks
 - turn the "bo" chunks into a sequence of alternating "punct" and "bo" chunks
 - turn the remaining "bo" chunks into a sequence of alternating "sym" and "bo" chunks
 - turn the remaining "bo" chunks into a sequence of alternating "num" and "bo" chunks
 - turn the remaining "bo" chunks into syllables
 - concatenate the chunks containing only spaces to the preceding one

The last pipe is created by using the building blocks of BoChunk, but implemented here, since this treatment merges two chunks given a condition, while BoChunk is a chunking framework.

In [5]:
from pybo import PyBoChunk

chunks = PyBoChunk(input_str)
output = chunks.chunk()

The raw output:

In [6]:
for n, chunk in enumerate(output):
    print(f'{n+1}.\t{chunk}')

1.	(102, 0, 2)
2.	(106, 2, 3)
3.	(106, 5, 4)
4.	(106, 9, 6)
5.	(101, 15, 3)
6.	(106, 18, 5)
7.	(106, 23, 5)
8.	(102, 28, 2)
9.	(106, 30, 4)
10.	(106, 34, 4)
11.	(106, 38, 4)
12.	(106, 42, 5)
13.	(109, 47, 3)
14.	(106, 50, 2)
15.	(102, 52, 2)
16.	(106, 54, 5)
17.	(106, 59, 4)
18.	(106, 63, 5)
19.	(106, 68, 4)
20.	(106, 72, 4)
21.	(106, 76, 3)
22.	(106, 79, 3)
23.	(106, 82, 5)
24.	(102, 87, 5)
25.	(106, 92, 3)
26.	(102, 95, 1)


The last two elements in the tuple are:
 - the starting index of the current chunk in the input string
 - the length of the current chunk
 
PyBoChunk provides a method to replace these two indices with the actual substring:

In [7]:
with_substrings = chunks.get_chunked(output)
for n, chunk in enumerate(with_substrings):
    print(f'{n+1}.\t{chunk}')

1.	(102, '༆ ')
2.	(106, 'ཤི་')
3.	(106, 'བཀྲ་')
4.	(106, 'ཤིས་  ')
5.	(101, 'tr ')
6.	(106, 'བདེ་་')
7.	(106, 'ལེ གས')
8.	(102, '། ')
9.	(106, 'བཀྲ་')
10.	(106, 'ཤིས་')
11.	(106, 'བདེ་')
12.	(106, 'ལེགས་')
13.	(109, '༡༢༣')
14.	(106, 'ཀཀ')
15.	(102, '། ')
16.	(106, 'མཐའི་')
17.	(106, 'རྒྱ་')
18.	(106, 'མཚོར་')
19.	(106, 'གནས་')
20.	(106, 'པའི་')
21.	(106, 'ཉས་')
22.	(106, 'ཆུ་')
23.	(106, 'འཐུང་')
24.	(102, '།། །།')
25.	(106, 'མཁའ')
26.	(102, '།')


I could do it manually, though:

In [8]:
chunk_str = input_str[output[0][1]:output[0][1]+output[0][2]]
print(f'"{with_substrings[0][1]}" equals "{chunk_str}"')

"༆ " equals "༆ "


Now, let's get human-readable values for the first int:

In [9]:
with_markers = chunks.get_markers(with_substrings)
for n, chunk in enumerate(with_markers):
    print(f'{n+1}.\t{chunk}')

1.	('punct', '༆ ')
2.	('syl', 'ཤི་')
3.	('syl', 'བཀྲ་')
4.	('syl', 'ཤིས་  ')
5.	('non-bo', 'tr ')
6.	('syl', 'བདེ་་')
7.	('syl', 'ལེ གས')
8.	('punct', '། ')
9.	('syl', 'བཀྲ་')
10.	('syl', 'ཤིས་')
11.	('syl', 'བདེ་')
12.	('syl', 'ལེགས་')
13.	('num', '༡༢༣')
14.	('syl', 'ཀཀ')
15.	('punct', '། ')
16.	('syl', 'མཐའི་')
17.	('syl', 'རྒྱ་')
18.	('syl', 'མཚོར་')
19.	('syl', 'གནས་')
20.	('syl', 'པའི་')
21.	('syl', 'ཉས་')
22.	('syl', 'ཆུ་')
23.	('syl', 'འཐུང་')
24.	('punct', '།། །།')
25.	('syl', 'མཁའ')
26.	('punct', '།')


 - punct: chunk containing Tibetan punctuation
 - syl: chunk containing a single syllable (necessarily Tibetan characters)
 - num: chunk containing Tibetan numerals
 - non-bo: chunk containing non-Tibetan characters
 
Note that at this point, the only thing we have is substrings of the initial string with chunk markers.

# bochunk.py

BoChunk is a chunking framework built on top of BoString (see below).

Its methods are used to create chunks – groups of characters that pertain to the same category. The produced output is a binary sequence of chunks matching or not a given condition.

Although each chunking method only produces two kinds of chunks (those that match and those that don't), piped chunking allows to apply complex chunking patterns.

In [7]:
from pybo.bochunk import BoChunk
    
bc = BoChunk(input_str)

## 1. The available chunking methods

#### Either Tibetan characters or not

In [10]:
bo_nonbo = bc.chunk_bo_chars()
print(f'the output of chunking functions are {type(bo_nonbo)} each containing {len(bo_nonbo[0])} {type(bo_nonbo[0][0])}')

the output of chunking functions are <class 'list'> each containing 3 <class 'int'>


In [8]:
for n, c in enumerate(chunks):
    print(f'{n+1}.\t{c}')

1.	(100, 0, 15)
2.	(101, 15, 2)
3.	(100, 17, 79)


We have already seen this format in PyBoChunk, which actually only applies the chunking methods available in BoChunk in order to produce chunks corresponding to the expectations of a Tibetan reader.

In a human-friendly format, it actually is:

In [14]:
bo_nonbo_readable = bc.get_markers(bc.get_chunked(bo_nonbo))
for n, r in enumerate(bo_nonbo_readable):
    print(f'{n+1}.\t{r[0]}\t\t"{r[1]}"')

1.	bo		"༆ ཤི་བཀྲ་ཤིས་  "
2.	non-bo		"tr"
3.	bo		" བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།"


We have indeed chunked the input string is a sequence of alternatively Tibetan and non-Tibetan chunks.

#### Either Tibetan punctuation or not

In [19]:
punct_nonpunct = bc.chunk_punct()
punct_nonpunct_readable = bc.get_markers(bc.get_chunked(punct_nonpunct))
for n, p in enumerate(punct_nonpunct_readable):
    print(f'{n+1}.\t{p[0]}\t\t"{p[1]}"')

1.	punct		"༆ "
2.	non-punct		"ཤི་བཀྲ་ཤིས་  tr"
3.	punct		" "
4.	non-punct		"བདེ་་ལེ གས"
5.	punct		"། "
6.	non-punct		"བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ"
7.	punct		"། "
8.	non-punct		"མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་"
9.	punct		"།། །།"
10.	non-punct		"མཁའ"
11.	punct		"།"


Now, we have an alternating sequence of chunks that only cares about Tibetan punctuation.

Note how "tr" is simply considered to be non-punct.

#### Either Tibetan symbols or not

In [22]:
sym_nonsym = bc.chunk_symbol()
sym_nonsym_readable = bc.get_markers(bc.get_chunked(sym_nonsym))
for n, s in enumerate(sym_nonsym_readable):
    print(f'{n+1}.\t{s[0]}\t\t"{s[1]}"')

1.	non-sym		"༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།"


... there were no Tibetan symbols in the input string.

#### Either Tibetan digits or not

In [27]:
num_nonnum = bc.chunk_number()
num_nonnum_readable = bc.get_markers(bc.get_chunked(num_nonnum))
for n, m in enumerate(num_nonnum_readable):
    print(f'{n+1}.\t{m[0]}\t\t"{m[1]}"')

1.	non-num		"༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་"
2.	num		"༡༢༣"
3.	non-num		"ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།"


We have correctly identified the Tibetan digits in the input string

#### Either spaces or not

In [28]:
space_nonspace = bc.chunk_spaces()
space_nonspace_readable = bc.get_markers(bc.get_chunked(space_nonspace))
for n, s in enumerate(space_nonspace_readable):
    print(f'{n+1}.\t{s[0]}\t\t"{s[1]}"')

1.	non-space		"༆"
2.	space		" "
3.	non-space		"ཤི་བཀྲ་ཤིས་"
4.	space		"  "
5.	non-space		"tr"
6.	space		" "
7.	non-space		"བདེ་་ལེ"
8.	space		" "
9.	non-space		"གས།"
10.	space		" "
11.	non-space		"བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ།"
12.	space		" "
13.	non-space		"མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།།"
14.	space		" "
15.	non-space		"།།མཁའ།"


#### Syllabify

In [29]:
syl_nonsyl = bc.syllabify()
syl_nonsyl_readable = bc.get_markers(bc.get_chunked(syl_nonsyl))
for n, s in enumerate(syl_nonsyl_readable):
    print(f'{n+1}.\t{s[0]}\t\t"{s[1]}"')

1.	syl		"༆ ཤི་"
2.	syl		"བཀྲ་"
3.	syl		"ཤིས་"
4.	syl		"  tr བདེ་་"
5.	syl		"ལེ གས། བཀྲ་"
6.	syl		"ཤིས་"
7.	syl		"བདེ་"
8.	syl		"ལེགས་"
9.	syl		"༡༢༣ཀཀ། མཐའི་"
10.	syl		"རྒྱ་"
11.	syl		"མཚོར་"
12.	syl		"གནས་"
13.	syl		"པའི་"
14.	syl		"ཉས་"
15.	syl		"ཆུ་"
16.	syl		"འཐུང་"
17.	syl		"།། །།མཁའ།"


This is the only chunking method that doesn't create a binary sequence of chunks matching or not a given condition. 

Instead, it breaks up the input it receives into syllables that are only separated by tseks(both regular and non-breaking). For this reason, syllabify takes for granted that the input it receives is only Tibetan characters that can compose a syllable.

When that was the case, for example in chunks 10 to 16, the syllabation is operated as expected.

Spaces, on the other hand, are allowed within a syllable because they only serve to "beautify" a tibetan text by visually marking the end of a clause or that of a sentence after a punctuation. So we reproduce the behavior of a Tibetan reader, who will simply bypass a space encountered anywhere else.

#### Extending the framework with additional methods

All of these chunking methods follow a simple standardized format, making it extremely simple to create new ones to suit any specific needs. They are in turn built on top of BoString, which relies on the groups of characters that were put together to reproduce the intuition of a native Tibetan who is reading a text.

So in case finer chunking abilities are required, finer character groups should be created in BoString.

## 2. Piped Chunking: Implementing complex chunking patterns

Applying successively the chunking methods we have seen above in a specific order allows to design complex patterns for chunking a given string.

We will exemplify this functionality by reproducing the chunking pipeline that PyBoChunk implements:

    a. turn the input string into a sequence of alternating "bo" / "non-bo" chunks
    b. turn the "bo" chunks into a sequence of alternating "punct" / "bo" chunks
    c. turn the remaining "bo" chunks into a sequence of alternating "sym" / "bo" chunks
    d. turn the remaining "bo" chunks into a sequence of alternating "num" / "bo" chunks
    e. turn the "bo" chunks into a sequence of syllables
    f. concatenate the chunks containing only spaces to the preceding one

#### a. Either "bo" or "non-bo"

In [53]:
chunks = bc.chunk_bo_chars()

print('\t', input_str, end='\n\n')
for n, c in enumerate(bc.get_markers(bc.get_chunked(chunks))):
    print(f'{n+1}.\t{c[0]}\t\t"{c[1]}"')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།

1.	bo		"༆ ཤི་བཀྲ་ཤིས་  "
2.	non-bo		"tr"
3.	bo		" བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།"


The output is similar to what we had above. Yet this time, instead of rechunking the whole input string in "punct" or "non-punct", we will use piped chunking to only apply chunk_punct() to the "bo" chunks.

Important: piped chunking is only possible when the content of the chunks is a tuple of ints.

#### b. Either "punct" or "bo"

In [54]:
bc.pipe_chunk(chunks, bc.chunk_punct, to_chunk=bc.BO_MARKER, yes=bc.PUNCT_MARKER)

print('\t', input_str, end='\n\n')
for n, c in enumerate(bc.get_markers(bc.get_chunked(chunks))):
    print(f'{n+1}.\t{c[0]}\t\t"{c[1]}"')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།

1.	punct		"༆ "
2.	bo		"ཤི་བཀྲ་ཤིས་  "
3.	non-bo		"tr"
4.	punct		" "
5.	bo		"བདེ་་ལེ གས"
6.	punct		"། "
7.	bo		"བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ"
8.	punct		"། "
9.	bo		"མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་"
10.	punct		"།། །།"
11.	bo		"མཁའ"
12.	punct		"།"


pipe_chunk() takes as arguments:
 - chunks: the chunks produced by a previous chunking methods. They are expected to be a list of tuples containing each three ints.
 - bc.chunk_punct: the chunking method we want to apply on top of the existing chunks
 - to_chunk: the marker that identifies which chunks should be further processed by the new chunking method
 - yes: the marker to be used on the new chunks that match the new chunking method. (the chunks that don't match simply retain their previous marker)

So to put it into simple English: 

                  "Within the existing chunks, I would like to take those marked as 'bo' 
                  and within them separate what is actual Tibetan text ('bo') 
                  from what is Tibetan punctuation ('punct')"

#### c. Either "sym" or "bo"

In [55]:
bc.pipe_chunk(chunks, bc.chunk_symbol, to_chunk=bc.BO_MARKER, yes=bc.SYMBOL_MARKER)

print('\t', input_str, end='\n\n')
for n, c in enumerate(bc.get_markers(bc.get_chunked(chunks))):
    print(f'{n+1}.\t{c[0]}\t\t"{c[1]}"')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།

1.	punct		"༆ "
2.	bo		"ཤི་བཀྲ་ཤིས་  "
3.	non-bo		"tr"
4.	punct		" "
5.	bo		"བདེ་་ལེ གས"
6.	punct		"། "
7.	bo		"བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ"
8.	punct		"། "
9.	bo		"མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་"
10.	punct		"།། །།"
11.	bo		"མཁའ"
12.	punct		"།"


There were no symbols, so the chunks are unchanged.

#### d. Either "num" or "bo"

In [56]:
bc.pipe_chunk(chunks, bc.chunk_number, to_chunk=bc.BO_MARKER, yes=bc.NUMBER_MARKER)

print('\t', input_str, end='\n\n')
for n, c in enumerate(bc.get_markers(bc.get_chunked(chunks))):
    print(f'{n+1}.\t{c[0]}\t\t"{c[1]}"')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།

1.	punct		"༆ "
2.	bo		"ཤི་བཀྲ་ཤིས་  "
3.	non-bo		"tr"
4.	punct		" "
5.	bo		"བདེ་་ལེ གས"
6.	punct		"། "
7.	bo		"བཀྲ་ཤིས་བདེ་ལེགས་"
8.	num		"༡༢༣"
9.	bo		"ཀཀ"
10.	punct		"། "
11.	bo		"མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་"
12.	punct		"།། །།"
13.	bo		"མཁའ"
14.	punct		"།"


                  "Within the existing chunks, I would like to take those marked as 'bo' 
                  and within them separate what is actual Tibetan text ('bo') 
                  from what is Tibetan digits ('num')"

#### e. Syllabify "bo" into "syl"

In [58]:
bc.pipe_chunk(chunks, bc.syllabify, to_chunk=bc.BO_MARKER, yes=bc.SYL_MARKER)

print('\t', input_str, end='\n\n')
for n, c in enumerate(bc.get_markers(bc.get_chunked(chunks))):
    print(f'{n+1}.\t{c[0]}\t\t"{c[1]}"')

	 ༆ ཤི་བཀྲ་ཤིས་  tr བདེ་་ལེ གས། བཀྲ་ཤིས་བདེ་ལེགས་༡༢༣ཀཀ། མཐའི་རྒྱ་མཚོར་གནས་པའི་ཉས་ཆུ་འཐུང་།། །།མཁའ།

1.	punct		"༆ "
2.	syl		"ཤི་"
3.	syl		"བཀྲ་"
4.	syl		"ཤིས་"
5.	syl		"  "
6.	non-bo		"tr"
7.	punct		" "
8.	syl		"བདེ་་"
9.	syl		"ལེ གས"
10.	punct		"། "
11.	syl		"བཀྲ་"
12.	syl		"ཤིས་"
13.	syl		"བདེ་"
14.	syl		"ལེགས་"
15.	num		"༡༢༣"
16.	syl		"ཀཀ"
17.	punct		"། "
18.	syl		"མཐའི་"
19.	syl		"རྒྱ་"
20.	syl		"མཚོར་"
21.	syl		"གནས་"
22.	syl		"པའི་"
23.	syl		"ཉས་"
24.	syl		"ཆུ་"
25.	syl		"འཐུང་"
26.	punct		"།། །།"
27.	syl		"མཁའ"
28.	punct		"།"


Here we are ! We have successfully preprocessed the input string into a sequence of identified content that we can confidently use for any further NLP processing.

In order to have the exact same output as PyBoChunk, chunk 5 should be attached to chunk 4 and chunk 7 to chunk 6. Since this is implemented as a private method of PyBoChunk, we will stop this demonstration here.

# bostring.py